# Проект для «Викишоп» c BERT

**Описание исследования**  

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.   
Необходимо создать модель, классифицирующую комментарии на позитивные и негативные. Для этого предоставлен набор данных с разметкой о токсичности правок. Значение метрики качества F1 должно быть не меньше 0.75. 

**Цель исследования**
- Создать модель оценки токсичности комментариев (да/нет)
- Метрика качества F1 должна быть не менее 0.75.

**Ход исследования**
- Обзор данных
- Предобработка данных
- Создание выборок
- Формирование пайплайнов
- Обучение моделей
- Проверка скорости предсказания
- Проверка на тестовой выборке
- Проверка константной моделью
- Общий вывод

## Импорт библиотек и загрузка данных

In [29]:
!python -m spacy download en_core_web_sm

     --------------------------------------- 12.8/12.8 MB 50.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
import pandas as pd
import numpy as np
import torch
import spacy


from tqdm import notebook
from tqdm import tqdm


from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import set_config

set_config(display="diagram")


from lightgbm import LGBMClassifier


import re


import nltk
from nltk.corpus import stopwords as nltk_stopwords


from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

In [31]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
# активация progress_apply
tqdm.pandas()

In [33]:
# RS для применения во всей тетрадке
RANDOM_STATE = 42

In [34]:
bert_tokenizer = AutoTokenizer.from_pretrained("s-nlp/roberta_toxicity_classifier")

In [35]:
bert_model = AutoModelForSequenceClassification.from_pretrained(
    "s-nlp/roberta_toxicity_classifier"
)

Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
try:
    df = pd.read_csv(
        r"C:\workspace\yandex_cources\full_history\sprint_12_intro_to_NLP\data\toxic_comments.csv"
    )
except:
    df = pd.read_csv(r"/datasets/toxic_comments.csv")

## Определение функций

In [37]:
def data_understanding(df):
    print("\n" * 2, "=" * 26, "5 случайных строк датасета", "=" * 27, "\n" * 2)
    display(df.sample(5))
    print("\n", "=" * 31, "Размеры датасета", "=" * 32, "\n" * 2)
    display(df.shape)
    print("\n" * 2, "=" * 29, "Информация о датасетe", "=" * 29, "\n" * 2)
    df.info()
    print(
        "\n" * 2, "=" * 21, "Информация о количественных признаках", "=" * 21, "\n" * 2
    )
    display(df.describe())

In [38]:
def toxicity_check(row):
    # prepare the input
    batch = bert_tokenizer.encode(row, return_tensors="pt")

    # inference
    out = bert_model(batch)

    # converting probabilities
    preds = torch.nn.functional.softmax(out.logits, dim=-1).detach().cpu().numpy()[0]

    # result
    if preds[0] > preds[1]:
        return 0
    else:
        return 1

In [39]:
def bert(series):
    # Заранее создадим словарь для исключений, когда длина токенизированной строки более 512 токенов и BERT не может с ней работать напрямую
    exceptions = {}
    bert_preds = []
    for i, row in notebook.tqdm(enumerate(series)):
        try:
            bert_preds.append(toxicity_check(row))
        except:
            # Чтобы BERT все же обработал строку, попавшую в исключения, разобъем ее на части, проверим каждую часть на токсичность и вернем 1, если хотя бы одна из частей окажется токсичной
            split_result = []
            for j in range(len(row.split()) // 128 + 1):
                n = len(row.split()) // (len(row.split()) // 128 + 1)
                row_slice = row[j * n : (j + 1) * n]
                split_result.append(toxicity_check(row_slice))

            # Для удобства сохраним результаты срезов и их общий результат в словарь, где ключом будет номер строки
            slices_result = f"Результаты срезов: {split_result}, финальный ответ: {max(split_result)}"
            exceptions[i] = slices_result

            bert_preds.append(max(split_result))

    return (bert_preds, exceptions)

## Обзор данных

In [40]:
data_understanding(df)



 ========================== 5 случайных строк датасета =========================== 




,Unnamed: 0,text,toxic
9870,9883,"""\n\nDo not assume or imply.\nIf reliable sour...",0
115547,115646,"""\n\n Info box image - """"current composition""""...",0
92210,92301,So by that logic I can never put anything rela...,0
43126,43176,SNL and Stewart are already mentioned in the S...,0
77214,77290,Meetup\nWikipedia:Meetup/Tampa You're invited!,0



 =============================== Размеры датасета ================================ 




(159292, 3)



 ============================= Информация о датасетe ============================= 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


 ===================== Информация о количественных признаках ===================== 




,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


В датасете осталась колонка старых индексов, которую можно удалить. 

In [41]:
df = df.drop(columns="Unnamed: 0")

## Предобработка данных

Избавимся от мусора в комментариях, оставим только слова.

In [42]:
def cleaner(row):
    return (" ".join(re.sub(r"[^a-zA-Z ]", " ", row).split())).lower()

In [43]:
df["preproc_text"] = df.text.progress_apply(lambda x: cleaner(x))

100%|██████████| 159292/159292 [00:02<00:00, 58534.78it/s]


In [44]:
nlp = spacy.load("en_core_web_sm")

In [45]:
def lemmatizer(series):
    lemmatized_text = []
    for batch in notebook.tqdm(
        nlp.pipe(series, batch_size=32, n_process=3, disable=["parser", "ner"])
    ):
        lemmatized_text.append(" ".join([tok.lemma_ for tok in batch]))
    return pd.DataFrame(lemmatized_text, index=series.index)

In [46]:
df["preproc_text"] = lemmatizer(df.preproc_text)

0it [00:00, ?it/s]

## Создание выборок

In [47]:
train, test = train_test_split(df, test_size=0.2, random_state=RANDOM_STATE)

In [48]:
X_train, X_test = train.preproc_text, test.preproc_text

In [49]:
y_train, y_test = train.toxic, test.toxic

## Формирование пайплайнов

In [50]:
stopwords = set(nltk_stopwords.words("english"))

In [51]:
lr_pipe = Pipeline(
    [
        (
            "tfidf_bow",
            FeatureUnion(
                [
                    ("tfidf_vec", TfidfVectorizer(stop_words=list(stopwords))),
                    ("bow", CountVectorizer(stop_words=list(stopwords))),
                ]
            ),
        ),
        ("lr", LogisticRegression()),
    ]
)

In [52]:
lgbm_pipe = Pipeline(
    [
        (
            "tfidf_bow",
            FeatureUnion(
                [
                    ("tfidf_vec", TfidfVectorizer(stop_words=list(stopwords))),
                    ("bow", CountVectorizer(stop_words=list(stopwords))),
                ]
            ),
        ),
        ("lgbm", LGBMClassifier()),
    ]
)

## Обучение моделей

### LogisticRegression

In [53]:
from sklearn.model_selection import GridSearchCV

In [54]:
lr_params = {
    "lr__C": [0.25, 0.5, 1, 2, 5],
    "lr__solver": ["liblinear"],
    "lr__max_iter": [2000],
}

In [55]:
lr_md = GridSearchCV(
    lr_pipe,
    lr_params,
    cv=5,
    scoring="f1",
    n_jobs=3,
)

In [56]:
%%time

lr_md.fit(X_train, y_train);

CPU times: total: 4min 9s
Wall time: 32min 53s


In [57]:
lr_md.best_params_

{'lr__C': 2, 'lr__max_iter': 2000, 'lr__solver': 'liblinear'}

In [58]:
print(
    "=" * 79,
    "\n\nLogistic Regression\n",
    f"\nF1 Train Score: {lr_md.best_score_:.3f}\n\n",
    "=" * 79,
)


Logistic Regression
 
F1 Train Score: 0.775



### LightGBM Classifier

In [59]:
from lightgbm import LGBMClassifier

In [60]:
lgbm_params = {
    "lgbm__n_estimators": [450, 600],
    "lgbm__max_depth": [15, 20, 25],
    "lgbm__class_weight": ["balanced"],
}

In [61]:
lgbm_md = GridSearchCV(
    lgbm_pipe,
    lgbm_params,
    cv=5,
    scoring="f1",
    n_jobs=3,
)

In [62]:
%%time

lgbm_md.fit(X_train, y_train);

CPU times: total: 20min 4s
Wall time: 31min 25s


In [63]:
lgbm_md.best_params_

{'lgbm__class_weight': 'balanced',
 'lgbm__max_depth': 15,
 'lgbm__n_estimators': 600}

In [64]:
print(
    "=" * 79,
    "\n\nLogistic Regression\n",
    f"\nF1 Train Score: {lgbm_md.best_score_:.3f}\n\n",
    "=" * 79,
)


Logistic Regression
 
F1 Train Score: 0.774



### BERT

In [65]:
X_train_cut = X_train.sample(5000, random_state=RANDOM_STATE)

In [66]:
y_train_cut = y_train.sample(5000, random_state=RANDOM_STATE)

In [67]:
bert_preds, exceptions = bert(X_train_cut)

0it [00:00, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (793 > 512). Running this sequence through the model will result in indexing errors


In [68]:
print(
    "=" * 79,
    "\n\nМодель на BERT\n",
    f"\nF1 Score: {f1_score(y_train_cut, bert_preds):.3f}\n\n",
    "=" * 79,
)


Модель на BERT
 
F1 Score: 0.841



Модели c TF-IDF/BOW пайплайном дали приблизительно равные результаты и достигли целевого показателя метрики F1, слегка перешагнув порог. BERT также достигла целевого показателя метрики F1, но при этом значительно превзошла обе TF-IDF/BOW модели. Посмотрим, сколько времени потребуется для выполнения 10 предсказаний каждой модели и выберем по скорости работы. 

## Проверка скорости предсказания

In [69]:
X_speed_test = X_test.sample(10, random_state=RANDOM_STATE)

In [70]:
%%timeit

lr_md.predict(X_speed_test)

4.92 ms ± 349 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [71]:
%%timeit

lgbm_md.predict(X_speed_test)

21.9 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [72]:
%%timeit

speed_preds, speed_excepts = bert(X_speed_test)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

1.99 s ± 46.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Самой быстрой оказалась модель LogisticRegression.

## Проверка на тестовой выборке

Поскольку все модели достигли целевого показателя F1, можно сделать выбор, исходя из скорости работы. Предположим, что в текущем проекте нам больше важна точность. Тогда лучшей моделью будет выбран BERT.

In [73]:
test_preds, out = bert(X_test.sample(1000, random_state=RANDOM_STATE))

0it [00:00, ?it/s]

In [74]:
print(
    "=" * 79,
    "\n\nМодель BERT на тестовой выборке\n",
    f"\nF1 Score: {f1_score(y_test.sample(1000, random_state=RANDOM_STATE), test_preds):.3f}\n\n",
    "=" * 79,
)


Модель BERT на тестовой выборке
 
F1 Score: 0.891



Целевое значение метрики достигнуто. 

## Проверка константной моделью

In [75]:
dummy_md_0 = DummyClassifier(strategy="constant", constant=0)

In [76]:
dummy_md_0.fit(X_train, y_train)

DummyClassifier(constant=0, strategy='constant')

In [77]:
dummy_preds_0 = dummy_md_0.predict(X_test)

In [78]:
print(
    "=" * 79,
    "\n\nDummy-модель (класс 0)\n",
    f"\nF1 Score: {f1_score(y_test, dummy_preds_0):.3f}\n\n",
    "=" * 79,
)


Dummy-модель (класс 0)
 
F1 Score: 0.000



In [79]:
dummy_md_1 = DummyClassifier(strategy="constant", constant=1)

In [80]:
dummy_md_1.fit(X_train, y_train)

DummyClassifier(constant=1, strategy='constant')

In [81]:
dummy_preds_1 = dummy_md_1.predict(X_test)

In [82]:
print(
    "=" * 79,
    "\n\nDummy-модель (класс 1)\n",
    f"\nF1 Score: {f1_score(y_test, dummy_preds_1):.3f}\n\n",
    "=" * 79,
)


Dummy-модель (класс 1)
 
F1 Score: 0.183



Метрика F1 константной модели значительно ниже выбранной основной модели. 

## Общий вывод

**В текущем исследовании** был рассмотрен датасет, содержащий выборку комментариев, для которых необходимо было построеть модель, определяющую их токсичность (да/нет). 

**Выполнены следующие этапы:**  
- **Обзор данных**  
    Изучено содержимое датасета, определено, что данные не требуют какой-либо обработки за исключением относящейся непосредственно к работе моделей.
    
- **Предобработка данных**  
    Текст комментариев приведен к нижнему регистру и очищен от любых символов, кроме латиницы, а также лемматизирован. 
    
- **Разбиение на выборки**  
    Датасет разделен на тренировочную и тестовую выборки
    
- **Построение TF-IDF/BOW моделей**   
    Созданы пайплайны для трансформации комментариев в наборы признако из TF-IDF матрицы и мешков слов, обучены модели LogisticRegression и LightGBM Classifier. 
   
- **Построение модели на основе BERT**  
    Создана функция, использующая предобученную модель BERT, определяющая является ли комментарий токсичным.
    
- **Проверка на тестовой выборке**  
    Лучшая модель, коей является модель, основанная на BERT, проверена на тестовой выборке; целевое значение метрики достигнуто. 
    
- **Проверка константной моделью**  
    Константная модель значительно отстает в качестве предсказаний от созданных моделей. 